In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from fastai.vision.all import *
from fastcore.utils import gt

In [3]:
path = Path("../.source")
path.ls(file_exts='.pkl')

(#1) [Path('../.source/export.pkl')]

In [4]:
learn_inf = load_learner(path/'export.pkl')

In [20]:
src = '../.source/test_data/spectro/'

In [23]:
night_preds = learn_inf.predict(src + 'night.png')
flower_preds = learn_inf.predict(src + 'flower.png')
tongue_preds = learn_inf.predict(src + 'tongue.png')


In [13]:
def create_intelligibility_score_lax(prediction, target_word):
  is_recognised = False
  score = 0.0

  ind = np.argpartition(prediction[2], -5)[-5:]
  if prediction[1] in ind:
    new_ind = np.delete(ind, [i for i in range(len(ind)) if ind[i] == prediction[1]])
    if prediction[0] == target_word:
      is_recognised = True
      score = prediction[2][prediction[1]] - prediction[2][new_ind][np.argmax(prediction[2][new_ind])]
      if score < 0.5: # for cases where the score would dip below .5, should be handled more elegantly
        score = prediction[2][prediction[1]]
    else:
      diff = (prediction[2][prediction[1]] - prediction[2][new_ind][np.argmax(prediction[2][new_ind])])
      if diff < 0.2 and prediction[2][prediction[1]] > 0.4:
        is_recognised = True
        score = prediction[2][prediction[1]]
      else:
        score = 0.5 - diff
      if score < 0:
        score = 0
  return is_recognised, score

In [25]:
print(create_intelligibility_score_lax(night_preds, 'night'))
print(create_intelligibility_score_lax(flower_preds, 'flower'))
print(create_intelligibility_score_lax(tongue_preds, 'tongue'))

(True, tensor(0.6508))
(False, 0)
(True, tensor(0.6594))
